In [4]:
!pip install requests

In [5]:
!pip install bs4

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    ElementClickInterceptedException,
    StaleElementReferenceException,
)
from selenium.webdriver.common.action_chains import ActionChains
import time
import random

In [ ]:
def fetch_css_selectors(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    css_selectors = []

    def extract_selectors(element):
        tag = element.name
        classes = element.get("class")
        class_selector = f".{'.'.join(classes)}" if classes else None
        id_selector = f"#{element.get('id')}" if element.get("id") else None

        if class_selector:
            css_selectors.append(class_selector)
        if id_selector:
            css_selectors.append(id_selector)
        if tag:
            css_selectors.append(tag)

        for child in element.children:
            if isinstance(child, BeautifulSoup) or isinstance(child, str):
                continue
            extract_selectors(child)

    extract_selectors(soup)
    return list(set(css_selectors))

In [ ]:
def handle_dynamic_popups(driver):
    try:
        potential_popups = driver.find_elements(By.CSS_SELECTOR, "*")
        for element in potential_popups:
            try:
                style = element.get_attribute("style")
                if "z-index" in style and "visibility: visible" in style:
                    close_buttons = element.find_elements(By.CSS_SELECTOR, "button, a, span")
                    for button in close_buttons:
                        if button.is_displayed() and button.is_enabled():
                            button.click()
                            time.sleep(1)
                            return True
                    action = ActionChains(driver)
                    random_x = random.randint(0, driver.execute_script("return window.innerWidth;"))
                    random_y = random.randint(0, driver.execute_script("return window.innerHeight;"))
                    action.move_by_offset(random_x, random_y).click().perform()
                    time.sleep(1)
                    return True
            except StaleElementReferenceException:
                continue
            except Exception:
                continue
    except Exception:
        pass
    return False

In [ ]:
def fetch_all_reviews(url, review_tag, author_tag, rating_tag, next_page_button_tag):
    service = Service("chromedriver.exe")  # Update this path if necessary
    driver = webdriver.Chrome(service=service)
    driver.maximize_window()

    try:
        driver.get(url)
        wait = WebDriverWait(driver, 10)
        all_reviews = []
        page_number = 1

        while True:
            try:
                wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, review_tag)))
                while handle_dynamic_popups(driver):
                    print("Pop-up dismissed. Retrying scraping...")

                review_elements = driver.find_elements(By.CSS_SELECTOR, review_tag)
                author_elements = driver.find_elements(By.CSS_SELECTOR, author_tag)
                rating_elements = driver.find_elements(By.CSS_SELECTOR, rating_tag)

                for review, author, rating in zip(review_elements, author_elements, rating_elements):
                    all_reviews.append({
                        "review": review.text.strip(),
                        "author": author.text.strip(),
                        "rating": rating.get_attribute("data-score") or "N/A",
                    })

                print(f"Page {page_number} scraped successfully with {len(review_elements)} reviews.")
                page_number += 1

                try:
                    next_page_button = driver.find_element(By.CSS_SELECTOR, next_page_button_tag)
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, next_page_button_tag))).click()
                    time.sleep(1)
                except (NoSuchElementException, TimeoutException):
                    print("No more pages to scrape or 'Next Page' button not found.")
                    break
                except ElementClickInterceptedException:
                    handle_dynamic_popups(driver)
                    continue

            except Exception as e:
                print(f"Error scraping page {page_number}: {str(e)}")
                break

    finally:
        driver.quit()

    return all_reviews

In [2]:
import requests
from bs4 import BeautifulSoup

# Fetch the webpage content
url = "https://2717recovery.com/products/recovery-cream"
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, "html.parser")

# Find all elements
elements = soup.find_all(True)  # 'True' means it finds all tags

# Initialize lists to store CSS selectors
css_selectors = []

# Loop through all elements
for element in elements:
    # Get the tag name
    tag = element.name
    
    # Get the class (if exists)
    classes = element.get("class")
    class_selector = f".{'.'.join(classes)}" if classes else None

    # Get the id (if exists)
    id_selector = f"#{element.get('id')}" if element.get("id") else None

    # Construct possible CSS selectors
    if class_selector:
        css_selectors.append(class_selector)
    if id_selector:
        css_selectors.append(id_selector)
    if tag:
        css_selectors.append(tag)

# Remove duplicates by converting the list to a set
unique_selectors = set(css_selectors)

# Print all unique CSS selectors
for selector in unique_selectors:
    print(selector)

body
.jdgm-rev__header
h4
.jdgm-paginate
.header.wrapper
.jdgm-rev__custom-form
#shopify-section-template--19759787049262__16600839769333e113
.newsletter-input
.template-product.template-product.no-touchevents.image-reveal--mask
.jdgm-medal.jdgm--loading
br
.product_details
#menu-open-button
#dynamic-checkout-cart
.product-variant__input.product-variant-value
.underline-animation
.jdgm-rev__buyer-badge-wrapper
.footer-item.footer-item--copyright
nav
.cart-continue
.jdgm-paginate__page.jdgm-curt
#shopify-section-header
.product__cart-actions.cart-actions.buttons-holder
.menu-meta__link
.footer-item.four-tenths.lap--full-width.footer-item--newsletter
.aria-hidden
#Email-popup
.form
img
.additional-checkout-buttons.additional-checkout-buttons--vertical
div
b
.primary-menu
.jdgm-rev__icon
.line
.jdgm-medals__container
.visually-hidden.skip-to-content
.jdgm-medal__value
.jdgm-rev__title
#ViewCart
#product-form-template--19759787049262__main
#cart
h5
#shopify-section-popup
.product__pricing


jdgm-rev__body<---review
jdgm-rev__author<---authors name

In [ ]:
import requests
from bs4 import BeautifulSoup

def fetch_reviews(url):
    # Fetch the webpage content
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code != 200:
        raise Exception(f"Failed to fetch the page. Status code: {response.status_code}")
    
    # Parse the HTML content
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Find all review elements
    review_elements = soup.find_all(class_="jdgm-rev__body")
    author_elements = soup.find_all(class_="jdgm-rev__author")
    rating_elements =soup.find_all(class_="jdgm-row-rating")
    
    # Extract reviews and authors
    reviews = []
    for review, author,rating in zip(review_elements, author_elements,rating_elements):
        review_text = review.get_text(strip=True)
        author_name = author.get_text(strip=True)
        reviews.append({"review": review_text, "author": author_name})
    
    return reviews

# URL of the webpage to scrape
url = "https://2717recovery.com/products/recovery-cream"

# Fetch and print reviews
reviews = fetch_reviews(url)
for idx, review in enumerate(reviews, 1):
    print(f"Review {idx}:")
    print(f"Author: {review['author']}")
    print(f"Review: {review['review']}")
    print("-" * 50)


Review 1:
Author: Shawna Churchill
Review: I love this stuff!
--------------------------------------------------
Review 2:
Author: Tania Patterson
Review: It’s amazing
--------------------------------------------------
Review 3:
Author: Francis Alvir
Review: So far so good . Still in trial stage
--------------------------------------------------
Review 4:
Author: G.P.
Review: So far so good
--------------------------------------------------
Review 5:
Author: Melvaree Witherspoon
Review: I absolutely love this product! I’ve been using it before my workouts, and it’s been a total game-changer. One of my favorite things about it is that it doesn’t smell like medicine—it has such a pleasant scent! I recently let my business partner use it on her knee that’s been giving her trouble, and she loved it just as much as I do. It’s effective, and enjoyable to use. I’ve even been sharing it on my social media because I believe in it so much. Great job, 27:17 team—thank you for creating such an ama

In [9]:
import requests
from bs4 import BeautifulSoup

def fetch_reviews(url):
    # Fetch the webpage content
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code != 200:
        raise Exception(f"Failed to fetch the page. Status code: {response.status_code}")
    
    # Parse the HTML content
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Find all review elements
    review_elements = soup.find_all(class_="jdgm-rev__body")
    author_elements = soup.find_all(class_="jdgm-rev__author")
    rating_elements = soup.find_all(class_="jdgm-rev__rating")
    
    # Extract reviews, authors, and ratings
    reviews = []
    for review, author, rating in zip(review_elements, author_elements, rating_elements):
        review_text = review.get_text(strip=True)
        author_name = author.get_text(strip=True)
        rating_score = rating.get("data-score", "N/A")  # Extract 'data-score' or use 'N/A' if missing
        reviews.append({
            "review": review_text,
            "author": author_name,
            "rating": rating_score
        })
    
    return reviews

# URL of the webpage to scrape
url = "https://2717recovery.com/products/recovery-cream"

# Fetch and print reviews
reviews = fetch_reviews(url)
for idx, review in enumerate(reviews, 1):
    print(f"Review {idx}:")
    print(f"Author: {review['author']}")
    print(f"Rating: {review['rating']} stars")
    print(f"Review: {review['review']}")
    print("-" * 50)


Review 1:
Author: Shawna Churchill
Rating: 5 stars
Review: I love this stuff!
--------------------------------------------------
Review 2:
Author: Tania Patterson
Rating: 4 stars
Review: It’s amazing
--------------------------------------------------
Review 3:
Author: Francis Alvir
Rating: 5 stars
Review: So far so good . Still in trial stage
--------------------------------------------------
Review 4:
Author: G.P.
Rating: 5 stars
Review: So far so good
--------------------------------------------------
Review 5:
Author: Melvaree Witherspoon
Rating: 5 stars
Review: I absolutely love this product! I’ve been using it before my workouts, and it’s been a total game-changer. One of my favorite things about it is that it doesn’t smell like medicine—it has such a pleasant scent! I recently let my business partner use it on her knee that’s been giving her trouble, and she loved it just as much as I do. It’s effective, and enjoyable to use. I’ve even been sharing it on my social media because I

In [10]:
!pip install selenium

   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.7 MB 6.4 MB/s eta 0:00:02
   --------- ------------------------------ 2.4/9.7 MB 5.7 MB/s eta 0:00:02
   ---------------- ----------------------- 3.9/9.7 MB 6.4 MB/s eta 0:00:01
   ----------------------- ---------------- 5.8/9.7 MB 6.9 MB/s eta 0:00:01
   ------------------------- -------------- 6.3/9.7 MB 6.3 MB/s eta 0:00:01
   ---------------------------------- ----- 8.4/9.7 MB 6.6 MB/s eta 0:00:01
   ---------------------------------------  9.7/9.7 MB 7.0 MB/s eta 0:00:01
   ---------------------------------------  9.7/9.7 MB 7.0 MB/s eta 0:00:01
   ---------------------------------------  9.7/9.7 MB 7.0 MB/s eta 0:00:01
   ---------------------------------------  9.7/9.7 MB 7.0 MB/s eta 0:00:01
   ---------------------------------------  9.7/9.7 MB 7.0 MB/s eta 0:00:01
   ---------------------------------------  9.7/9.7 MB 7.0 MB/s eta 0:00:01
   ----------------

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
import time

def fetch_all_reviews(url):
    # Set up Selenium WebDriver
    service = Service("C:/Users/Anuj Dwivedi/python api/chromedriver.exe")  # Update this path
    driver = webdriver.Chrome(service=service)
    driver.maximize_window()  # Maximize window for better visibility

    try:
        # Open the webpage
        driver.get(url)
        wait = WebDriverWait(driver, 10)  # Wait for elements to load

        all_reviews = []
        page_number = 1  # Track the current page number for debugging

        while True:
            try:
                # Wait for the reviews section to load
                wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "jdgm-rev__body")))

                # Get the current page source and parse it with BeautifulSoup
                soup = BeautifulSoup(driver.page_source, "html.parser")

                # Find all review elements
                review_elements = soup.find_all(class_="jdgm-rev__body")
                author_elements = soup.find_all(class_="jdgm-rev__author")
                rating_elements = soup.find_all(class_="jdgm-rev__rating")

                # Extract reviews, authors, and ratings
                for review, author, rating in zip(review_elements, author_elements, rating_elements):
                    review_text = review.get_text(strip=True)
                    author_name = author.get_text(strip=True)
                    rating_score = rating.get("data-score", "N/A")  # Extract 'data-score' or use 'N/A' if missing
                    all_reviews.append({
                        "review": review_text,
                        "author": author_name,
                        "rating": rating_score
                    })

                print(f"Page {page_number} scraped successfully with {len(review_elements)} reviews.")
                page_number += 1

                # Scroll the "Next Page" button into view and click it
                try:
                    next_page_button = driver.find_element(By.CSS_SELECTOR, "a.jdgm-paginate__next-page")
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.jdgm-paginate__next-page"))).click()
                    time.sleep(2)  # Small pause to ensure the page loads
                except (NoSuchElementException, TimeoutException):
                    print("No more pages to scrape or 'Next Page' button not found.")
                    break
                except ElementClickInterceptedException as e:
                    print(f"Click intercepted on page {page_number}: {str(e)}")
                    ActionChains(driver).move_to_element_with_offset(next_page_button, 0, 0).click().perform()
            except Exception as e:
                print(f"Error scraping page {page_number}: {str(e)}")
                break

    finally:
        # Ensure the browser is closed properly
        driver.quit()

    return all_reviews


# URL of the webpage to scrape
url = "https://2717recovery.com/products/recovery-cream"

# Fetch and print reviews
if __name__ == "__main__":
    reviews = fetch_all_reviews(url)
    for idx, review in enumerate(reviews, 1):
        print(f"Review {idx}:")
        print(f"Author: {review['author']}")
        print(f"Rating: {review['rating']} stars")
        print(f"Review: {review['review']}")
        print("-" * 50)


Page 1 scraped successfully with 5 reviews.
Click intercepted on page 2: Message: element click intercepted: Element <a class="jdgm-paginate__page jdgm-paginate__next-page" data-page="2" aria-label="Page 2" tabindex="0" rel="next" role="button"></a> is not clickable at point (664, 15). Other element would receive the click: <div role="dialog" aria-modal="true" aria-label="POPUP Form" class="needsclick  kl-private-reset-css-Xuajs1" style="display: flex; z-index: 90000; position: fixed; left: 0px; top: 0px; width: 100%; height: 100%; justify-content: center; align-items: center; overflow: clip auto; background-color: rgba(20, 20, 20, 0.6); animation-timing-function: ease; animation-play-state: running; animation-iteration-count: 1; animation-fill-mode: forwards; animation-delay: 0s; animation-duration: 0.35s; animation-name: klaviyo-fadein;">...</div>
  (Session info: chrome=131.0.6778.265)
Stacktrace:
	GetHandleVerifier [0x00007FF7529280D5+2992373]
	(No symbol) [0x00007FF7525BBFD0]
	(No

In [15]:
import requests
from bs4 import BeautifulSoup

def extract_text_from_url(url):
    try:
        # Send an HTTP GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an error if the request fails

        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, "html.parser")

        # Extract all text from the webpage
        page_text = soup.get_text()

        # Clean up the text by removing excessive whitespace
        clean_text = "\n".join(line.strip() for line in page_text.splitlines() if line.strip())

        return clean_text
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# URL of the webpage to extract text from
url = "https://2717recovery.com/products/recovery-cream"

# Extract and print the text
text = extract_text_from_url(url)
if text:
    print(text)


27:17 | Relief & Recovery Cream
Skip to content
0
Close
Back
Home
Shop Now
Our Mission
Ambassador Application
Wholesale Partnerships
Close
0 products in your cart
0
Total: $0.00
Shipping & taxes calculated at checkout
View Cart
Update cart
Checkout          One or more of the items in your cart is a recurring or deferred purchase. By continuing, I agree to the cancellation policy and authorize you to charge my payment method at the prices, frequency and dates listed on this page until my order is fulfilled or I cancel, if permitted.
Continue browsing
420 reviews
[BACKORDER] Relief & Recovery Cream
100 ml / 3.4 oz
📦 All backorders will ship January 17th 📦
NEW IMPROVED FORMULA COMING JANUARY 17TH
We’ve updated our formula for even faster absorption and results. Click below to view full ingredients list.
Description
A pain-relief cream created for high-performance athletes that helps speed up the body's natural recovery process and remove the source of discomfort. Now available for everyb

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
import time

# Step 1: Function to fetch and extract tags from a webpage
def extract_css_selectors(url):
    # Fetch the webpage content
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # Find all elements
    elements = soup.find_all(True)  # 'True' means it finds all tags

    # Initialize lists to store CSS selectors
    css_selectors = []

    # Loop through all elements
    for element in elements:
        # Get the tag name
        tag = element.name

        # Get the class (if exists)
        classes = element.get("class")
        class_selector = f".{'.'.join(classes)}" if classes else None

        # Get the id (if exists)
        id_selector = f"#{element.get('id')}" if element.get("id") else None

        # Construct possible CSS selectors
        if class_selector:
            css_selectors.append(class_selector)
        if id_selector:
            css_selectors.append(id_selector)
        if tag:
            css_selectors.append(tag)

    # Remove duplicates by converting the list to a set
    unique_selectors = set(css_selectors)

    return unique_selectors

# Step 2: Generalized function to scrape reviews using extracted tags and URL
def fetch_all_reviews(url, review_tag, author_tag, rating_tag):
    # Set up Selenium WebDriver
    service = Service("C:/Users/Anuj Dwivedi/python api/chromedriver.exe")  # Update this path
    driver = webdriver.Chrome(service=service)
    driver.maximize_window()  # Maximize window for better visibility

    try:
        # Open the webpage
        driver.get(url)
        wait = WebDriverWait(driver, 10)  # Wait for elements to load

        all_reviews = []
        page_number = 1  # Track the current page number for debugging

        while True:
            try:
                # Wait for the reviews section to load
                wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, review_tag)))

                # Get the current page source and parse it with BeautifulSoup
                soup = BeautifulSoup(driver.page_source, "html.parser")

                # Find all review elements using the passed review_tag, author_tag, and rating_tag
                review_elements = soup.select(review_tag)
                author_elements = soup.select(author_tag)
                rating_elements = soup.select(rating_tag)

                # Extract reviews, authors, and ratings
                for review, author, rating in zip(review_elements, author_elements, rating_elements):
                    review_text = review.get_text(strip=True)
                    author_name = author.get_text(strip=True)
                    rating_score = rating.get("data-score", "N/A")  # Extract 'data-score' or use 'N/A' if missing
                    all_reviews.append({
                        "review": review_text,
                        "author": author_name,
                        "rating": rating_score
                    })

                print(f"Page {page_number} scraped successfully with {len(review_elements)} reviews.")
                page_number += 1

                # Scroll the "Next Page" button into view and click it
                try:
                    next_page_button = driver.find_element(By.CSS_SELECTOR, "a.jdgm-paginate__next-page")
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.jdgm-paginate__next-page"))).click()
                    time.sleep(2)  # Small pause to ensure the page loads
                except (NoSuchElementException, TimeoutException):
                    print("No more pages to scrape or 'Next Page' button not found.")
                    break
                except ElementClickInterceptedException as e:
                    print(f"Click intercepted on page {page_number}: {str(e)}")
                    ActionChains(driver).move_to_element_with_offset(next_page_button, 0, 0).click().perform()
            except Exception as e:
                print(f"Error scraping page {page_number}: {str(e)}")
                break

    finally:
        # Ensure the browser is closed properly
        driver.quit()

    return all_reviews

# Step 3: Use the above functions with dynamic parameters

# URL of the webpage to scrape
url = "https://2717recovery.com/products/recovery-cream"

# Extract CSS selectors for the review section, author, and rating from the URL
unique_selectors = extract_css_selectors(url)

# These could be dynamic, so let's assume we know which CSS selectors to use for this specific site
review_tag = ".jdgm-rev__body"
author_tag = ".jdgm-rev__author"
rating_tag = ".jdgm-rev__rating"

# Fetch and print reviews
if __name__ == "__main__":
    reviews = fetch_all_reviews(url, review_tag, author_tag, rating_tag)
    for idx, review in enumerate(reviews, 1):
        print(f"Review {idx}:")
        print(f"Author: {review['author']}")
        print(f"Rating: {review['rating']} stars")
        print(f"Review: {review['review']}")
        print("-" * 50)


Page 1 scraped successfully with 5 reviews.
Click intercepted on page 2: Message: element click intercepted: Element <a class="jdgm-paginate__page jdgm-paginate__next-page" data-page="2" aria-label="Page 2" tabindex="0" rel="next" role="button"></a> is not clickable at point (664, 15). Other element would receive the click: <div role="dialog" aria-modal="true" aria-label="POPUP Form" class="needsclick  kl-private-reset-css-Xuajs1" style="display: flex; z-index: 90000; position: fixed; left: 0px; top: 0px; width: 100%; height: 100%; justify-content: center; align-items: center; overflow: clip auto; background-color: rgba(20, 20, 20, 0.6); animation-timing-function: ease; animation-play-state: running; animation-iteration-count: 1; animation-fill-mode: forwards; animation-delay: 0s; animation-duration: 0.35s; animation-name: klaviyo-fadein;">...</div>
  (Session info: chrome=131.0.6778.265)
Stacktrace:
	GetHandleVerifier [0x00007FF7F64580D5+2992373]
	(No symbol) [0x00007FF7F60EBFD0]
	(No

In [14]:
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    ElementClickInterceptedException,
    StaleElementReferenceException,
)
import time
import random


# Function to handle all pop-ups dynamically, including random clicks
def handle_dynamic_popups(driver):
    try:
        potential_popups = driver.find_elements(By.CSS_SELECTOR, "*")
        for element in potential_popups:
            try:
                style = element.get_attribute("style")
                if "z-index" in style and "visibility: visible" in style:
                    close_buttons = element.find_elements(By.CSS_SELECTOR, "button, a, span")
                    for button in close_buttons:
                        if button.is_displayed() and button.is_enabled():
                            button.click()
                            time.sleep(1)
                            return True
                    action = ActionChains(driver)
                    random_x = random.randint(0, driver.execute_script("return window.innerWidth;"))
                    random_y = random.randint(0, driver.execute_script("return window.innerHeight;"))
                    action.move_by_offset(random_x, random_y).click().perform()
                    time.sleep(1)
                    return True
            except StaleElementReferenceException:
                continue
            except Exception:
                continue
    except Exception:
        pass
    return False


# Function to scrape reviews
def fetch_all_reviews(url, review_tag, author_tag, rating_tag, next_page_button_tag):
    service = Service("chromedriver.exe")  # Update this path
    driver = webdriver.Chrome(service=service)
    driver.maximize_window()

    try:
        driver.get(url)
        wait = WebDriverWait(driver, 10)
        all_reviews = []
        page_number = 1

        while True:
            try:
                wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, review_tag)))
                while handle_dynamic_popups(driver):
                    print("Pop-up dismissed. Retrying scraping...")

                review_elements = driver.find_elements(By.CSS_SELECTOR, review_tag)
                author_elements = driver.find_elements(By.CSS_SELECTOR, author_tag)
                rating_elements = driver.find_elements(By.CSS_SELECTOR, rating_tag)

                for review, author, rating in zip(review_elements, author_elements, rating_elements):
                    all_reviews.append({
                        "review": review.text.strip(),
                        "author": author.text.strip(),
                        "rating": rating.get_attribute("data-score") or "N/A",
                    })

                print(f"Page {page_number} scraped successfully with {len(review_elements)} reviews.")
                page_number += 1

                try:
                    next_page_button = driver.find_element(By.CSS_SELECTOR, next_page_button_tag)
                    driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, next_page_button_tag))).click()
                    time.sleep(1)
                except (NoSuchElementException, TimeoutException):
                    print("No more pages to scrape or 'Next Page' button not found.")
                    break
                except ElementClickInterceptedException:
                    handle_dynamic_popups(driver)
                    continue

            except Exception as e:
                print(f"Error scraping page {page_number}: {str(e)}")
                break

    finally:
        driver.quit()

    return all_reviews


# URL of the webpage to scrape
url = "https://2717recovery.com/products/recovery-cream"

# CSS Selectors for this specific site
review_tag = ".jdgm-rev__body"
author_tag = ".jdgm-rev__author"
rating_tag = ".jdgm-rev__rating"
next_page_button_tag = "a.jdgm-paginate__page.jdgm-paginate__next-page"


# Fetch and save reviews in JSON
if __name__ == "__main__":
    reviews = fetch_all_reviews(url, review_tag, author_tag, rating_tag, next_page_button_tag)
    with open("reviews.json", "w", encoding="utf-8") as file:
        json.dump(reviews, file, ensure_ascii=False, indent=4)
    print(f"Saved {len(reviews)} reviews to 'reviews.json'.")


Page 1 scraped successfully with 5 reviews.
Page 2 scraped successfully with 5 reviews.
Page 3 scraped successfully with 5 reviews.
Error scraping page 4: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.265)
Stacktrace:
	GetHandleVerifier [0x00007FF685B480D5+2992373]
	(No symbol) [0x00007FF6857DBFD0]
	(No symbol) [0x00007FF68567590A]
	(No symbol) [0x00007FF68564F4F5]
	(No symbol) [0x00007FF6856F63A7]
	(No symbol) [0x00007FF68570EE72]
	(No symbol) [0x00007FF6856EF113]
	(No symbol) [0x00007FF6856BA918]
	(No symbol) [0x00007FF6856BBA81]
	GetHandleVerifier [0x00007FF685BA6A2D+3379789]
	GetHandleVerifier [0x00007FF685BBC32D+3468109]
	GetHandleVerifier [0x00007FF685BB0043+3418211]
	GetHandleVerifier [0x00007FF68593C78B+847787]
	(No symbol) [0x00007FF6857E757F]
	(No symbol) [0x00007FF6857E2FC4]
	(No symbol) [0x00007FF6857E315D]
	(No symbol) [0x00007FF6857D2979]
	BaseThreadInitThunk [0x00007FFFC2A2259D+29]
	RtlUser

In [19]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Configure Chrome options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")  # Optional: Start browser maximized
service = Service("chromedriver.exe")  # Replace with the correct path to your chromedriver
driver = webdriver.Chrome(service=service, options=chrome_options)

# Open the website
url = "https://2717recovery.com/products/recovery-cream"  # Replace with your target website
driver.get(url)

# Scroll to the bottom of the page
scroll_pause_time = 2  # Time to wait between scrolls in seconds
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to the bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)  # Wait for page to load after scrolling
    
    # Calculate new scroll height and compare with last height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:  # If the height is the same, we've reached the bottom
        print("Reached the bottom of the page.")
        break
    last_height = new_height
    tim

# Close the browser
driver.quit()


Reached the bottom of the page.


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Function to extract unique tags
def extract_unique_tags(url):
    # Set up Selenium WebDriver
    service = Service("chromedriver.exe")  # Update this path
    driver = webdriver.Chrome(service=service)
    driver.maximize_window()

    try:
        # Open the webpage
        driver.get(url)
        wait = WebDriverWait(driver, 10)

        # Wait until the page loads
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "html")))

        # Find all elements in the page
        elements = driver.find_elements(By.XPATH, "//*")

        # Extract unique tags
        unique_tags = set()
        for element in elements:
            tag = element.tag_name
            unique_tags.add(tag)

        # Return the unique tags
        return unique_tags

    finally:
        driver.quit()

# URL of the website to scrape
url = "https://www.example.com"  # Replace with your desired URL

# Fetch unique tags from the webpage
if __name__ == "__main__":
    unique_tags = extract_unique_tags(url)
    print("Unique tags on the page:")
    for tag in unique_tags:
        print(tag)


In [9]:
import random
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

# Configure Selenium WebDriver
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
service = Service("chromedriver.exe")  # Update with the correct path
driver = webdriver.Chrome(service=service, options=chrome_options)

# Open the website
url = "https://2717recovery.com/products/recovery-cream"
driver.get(url)

# Function to randomly click on the screen
def random_screen_click(driver):
    # Get the window size
    window_width = driver.execute_script("return window.innerWidth")
    window_height = driver.execute_script("return window.innerHeight")

    # Generate random x, y coordinates
    x = random.randint(0, window_width - 1)
    y = random.randint(0, window_height - 1)

    # Use JavaScript to simulate the click
    driver.execute_script(f"document.elementFromPoint({x}, {y}).click();")
    print(f"Clicked at random coordinates: ({x}, {y})")

# Check for popups and randomly click
try:
    time.sleep(25)  # Wait for potential popups to load
    # Example: Detecting a specific popup (adjust selector to your use case)
    popup_detected = driver.find_element(By.CSS_SELECTOR, ".popup-class")  # Update with the actual selector

    if popup_detected:
        print("Popup detected, attempting to close it by random click.")
        random_screen_click(driver)

        # Verify if the popup is closed
        try:
            time.sleep(2)  # Allow time for the click to take effect
            driver.find_element(By.CSS_SELECTOR, ".popup-class")  # Check if popup still exists
            print("Popup not closed.")
        except NoSuchElementException:
            print("Popup closed successfully.")
except NoSuchElementException:
    print("No popup detected.")

# Close the browser
driver.quit()


No popup detected.


In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Function to extract unique tags
def extract_unique_tags(url):
    # Set up Selenium WebDriver
    service = Service("chromedriver.exe")  # Update this path
    driver = webdriver.Chrome(service=service)
    driver.maximize_window()

    try:
        # Open the webpage
        driver.get(url)
        wait = WebDriverWait(driver, 10)

        # Wait for the body or reviews section to ensure the page is loaded
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

        # Ensure that the reviews section is visible
        # For Amazon, the review section can be identified by a specific tag
        try:
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-asin]")))
        except Exception as e:
            print("Error while waiting for reviews section:", e)

        # Find all elements on the page
        elements = driver.find_elements(By.XPATH, "//*")

        # Extract unique tags
        unique_tags = set()
        for element in elements:
            tag = element.tag_name
            unique_tags.add(tag)

        # Return the unique tags
        return unique_tags

    finally:
        driver.quit()

# URL of the website to scrape
url = "https://www.amazon.in/Puma-Mens-ATTACANTO-Black-Poison-Football/dp/B0C2V6GNZS/ref=asc_df_B0C2V6GNZS/?tag=googleshopdes-21&linkCode=df0&hvadid=710066525906&hvpos=&hvnetw=g&hvrand=9537220491062594138&hvpone=&hvptwo=&hvqmt=&hvdev=c&hvdvcmdl=&hvlocint=&hvlocphy=9062061&hvtargid=pla-2283327688705&mcid=6482f7f7a59f3914adbe2c782f60512c&gad_source=1&th=1&psc=1"  # Replace with your desired URL

# Fetch unique tags from the webpage
if __name__ == "__main__":
    unique_tags = extract_unique_tags(url)
    print("Unique tags on the page:")
    for tag in unique_tags:
        print(tag)


Error while waiting for reviews section: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6BFF180D5+2992373]
	(No symbol) [0x00007FF6BFBABFD0]
	(No symbol) [0x00007FF6BFA4590A]
	(No symbol) [0x00007FF6BFA9926E]
	(No symbol) [0x00007FF6BFA9955C]
	(No symbol) [0x00007FF6BFAE27D7]
	(No symbol) [0x00007FF6BFABF3AF]
	(No symbol) [0x00007FF6BFADF584]
	(No symbol) [0x00007FF6BFABF113]
	(No symbol) [0x00007FF6BFA8A918]
	(No symbol) [0x00007FF6BFA8BA81]
	GetHandleVerifier [0x00007FF6BFF76A2D+3379789]
	GetHandleVerifier [0x00007FF6BFF8C32D+3468109]
	GetHandleVerifier [0x00007FF6BFF80043+3418211]
	GetHandleVerifier [0x00007FF6BFD0C78B+847787]
	(No symbol) [0x00007FF6BFBB757F]
	(No symbol) [0x00007FF6BFBB2FC4]
	(No symbol) [0x00007FF6BFBB315D]
	(No symbol) [0x00007FF6BFBA2979]
	BaseThreadInitThunk [0x00007FFFC2A2259D+29]
	RtlUserThreadStart [0x00007FFFC2C2AF38+40]

Unique tags on the page:
link
h4
span
title
html
meta
noscript
script
body
input
div
img
button
form
head
a
i
p
